In [4]:
import imageio
import numpy as np
from argparse import ArgumentParser

import torch

from trainer import Trainer
from utils.tools import get_config
import easydict

In [5]:
args = easydict.EasyDict({ 
    "config": "configs/config.yaml", 
    "image": '', 
    "mask": '', 
    "output": "output.png", 
    "model_path": '', 
})

args.image = "examples/imagenet/imagenet_patches_ILSVRC2012_val_00008210_input.png"
args.mask = "examples/center_mask_256.png"
args.output = "examples/output.png"
args.model_path = "torch_model.p"

In [6]:
def main():
    config = get_config(args.config)
    if config['cuda']:
        device = torch.device("cuda:{}".format(config['gpu_ids'][0]))
    else:
        device = torch.device("cpu")
    trainer = Trainer(config)
    trainer.load_state_dict(load_weights(args.model_path, device), strict=False)
    trainer.eval()

    image = imageio.imread(args.image)
    image = torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0).cuda()
    mask = imageio.imread(args.mask)
    mask = (torch.FloatTensor(mask[:, :, 0]) / 255).unsqueeze(0).unsqueeze(0).cuda()

    x = (image / 127.5 - 1) * (1 - mask).cuda()
    with torch.no_grad():
        _, result, _ = trainer.netG(x, mask)

    imageio.imwrite(args.output, upcast(result[0].permute(1, 2, 0).detach().cpu().numpy()))


def load_weights(path, device):
    model_weights = torch.load(path)
    return {
        k: v.to(device)
        for k, v in model_weights.items()
    }


def upcast(x):
    return np.clip((x + 1) * 127.5 , 0, 255).astype(np.uint8)


In [7]:
main()

/home/dajinhan/workspace/generative-inpainting-pytorch/utils/tools.py:476: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)
/home/dajinhan/anaconda3/envs/pytorch-1.7/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
